In [78]:
# example of grid searching key hyperparametres for ridge classifier
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import scipy
import re


In [79]:

# import dataset
dataset = pd.read_csv('csv/cleaned_train_all.csv')

testdataset = pd.read_csv('csv/cleaned_test_all.csv')



x = dataset.drop(['credit_card_default'], axis = 1)
y = dataset['credit_card_default'].values
print(x.shape, y.shape)

(43508, 16) (43508,)


In [80]:
scalar =  preprocessing.StandardScaler()

# copy of datasets
X_train = x.copy()

# numerical features

#for all 
num_cols = ['net_yearly_income','no_of_days_employed','yearly_debt_payments',
             'credit_limit',"credit_limit_used(%)", "credit_score"]

#for less
num_cols = ["credit_limit_used(%)", "credit_score"]

#apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = scalar.fit(X_train[[i]])
    scale2 =  scalar.fit(testdataset[[i]])
    # transform the training data column
    X_train[i] = scale.transform(X_train[[i]])
    testdataset[i] = scale.transform(testdataset[[i]])

X_train = X_train.set_index('customer_id')
testdataset = testdataset.set_index('customer_id')



In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [82]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size = 0.25, random_state = 0)



In [83]:
adaboost = AdaBoostClassifier(base_estimator = None, n_estimators = 200, learning_rate = 1.0)
adaboost.fit(X_train, y_train)


AdaBoostClassifier(n_estimators=200)

In [87]:
gbdt = GradientBoostingClassifier(loss = 'deviance', learning_rate = 0.1, n_estimators = 200, min_samples_split = 2,
                                  min_samples_leaf = 1, max_depth = 3, subsample = 1.0)

gbdt.fit(X_train, y_train)


GradientBoostingClassifier(n_estimators=200)

In [84]:
adaParams = {'n_estimators':[10,50,100,200,400]}

from sklearn.model_selection import RandomizedSearchCV
ada = RandomizedSearchCV(estimator = adaboost, param_distributions = adaParams, n_iter = 5, scoring = 'roc_auc',
                          cv = None, verbose = 2).fit(X_train, y_train)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=50; total time=   1.0s
[CV] END ....................................n_estimators=50; total time=   1.0s
[CV] END ....................................n_estimators=50; total time=   1.1s
[CV] END ....................................n_estimators=50; total time=   1.1s
[CV] END ....................................n_estimators=50; total time=   1.1s
[CV] END ...................................n_estimators=100; total time=   2.2s
[CV] END ...................................n_est

In [85]:
y_pred_ADAH =ada.predict(X_test)


In [86]:
print('F1 Score - :', metrics.f1_score(y_test, y_pred_ADAH , average="macro" ) )  


F1 Score - : 0.9182317250459042


In [89]:
# Hyper parameter tuning for GBDT :
from random import randint
gbParams = {'loss': ['deviance', 'exponential'],
            'n_estimators': [10, 50, 100, 200, 400],
            'max_depth': [3,5,7,9]}


gb = RandomizedSearchCV(estimator = gbdt, param_distributions = gbParams, n_iter = 10, scoring = 'roc_auc',
                       cv = None, verbose = 2).fit(X_train, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ........loss=deviance, max_depth=9, n_estimators=50; total time=   3.7s
[CV] END ........loss=deviance, max_depth=9, n_estimators=50; total time=   3.8s
[CV] END ........loss=deviance, max_depth=9, n_estimators=50; total time=   3.5s
[CV] END ........loss=deviance, max_depth=9, n_estimators=50; total time=   4.1s
[CV] END ........loss=deviance, max_depth=9, n_estimators=50; total time=   3.7s
[CV] END ....loss=exponential, max_depth=9, n_estimators=100; total time=   9.7s
[CV] END ....loss=exponential, max_depth=9, n_estimators=100; total time=   9.6s
[CV] END ....loss=exponential, max_depth=9, n_estimators=100; total time=   9.4s
[CV] END ....loss=exponential, max_depth=9, n_estimators=100; total time=   9.9s
[CV] END ....loss=exponential, max_depth=9, n_estimators=100; total time=   9.8s
[CV] END ........loss=deviance, max_depth=3, n_estimators=10; total time=   0.3s
[CV] END ........loss=deviance, max_depth=3, n_e

In [90]:
y_pred_GBDTH =ada.predict(X_test)
print('F1 Score - :', metrics.f1_score(y_test, y_pred_GBDTH , average="macro" ) )  


F1 Score - : 0.9182317250459042
